In [1]:
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = 'Cuda_prac/stereo_depth'

%cd /content/drive/My\ Drive/$FOLDERNAME/

Mounted at /content/drive
/content/drive/My Drive/Cuda_prac/stereo_depth


In [2]:
from util import Dataset, toHomogeneous
from google.colab.patches import cv2_imshow
from calibration import feature_matching, estimate_F_RANSAC
import cv2
import numpy as np

dataset = Dataset('curule')

h,w = dataset.img0.shape[0:2]


In [3]:
!pip install pycuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 50.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 KB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 12.1 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp38-cp38-linux_x86_64.whl size=661775 sha256=9f924a1e5c9c8bfd03880d86cfd914a71b4ee87f8ffbf7cbe0edd931484827f2
  Stored in directory: /root/.cache/pip/wheels/7b/41/0d/7cecb04af969d283ebe4a69579a8b2baec0d010a1ac4159f7e
  Created wheel for pytools: filename=pytools-2022.1.14-py2.py3-none-any.whl size=69870 sha256=02c01b88984d46e09532fdef9899774e4dea4bfbb46aca7a5008df2efcb3f0fd
  Stored in directory: /root/.cache/pip/wheels/cb/fc/

In [4]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [5]:
mod = SourceModule(open('kernels.cu', 'r').read())

In [8]:
gray_img = cv2.cvtColor(dataset.img0, cv2.COLOR_BGR2GRAY)
gray_img = gray_img.astype(np.float32)
res_img_buffer = np.zeros_like(gray_img)

gray_gpu = cuda.mem_alloc(gray_img.nbytes)
res_gpu = cuda.mem_alloc(res_img_buffer.nbytes)

cuda.memcpy_htod(gray_gpu, gray_img)
cuda.memcpy_htod(res_gpu, res_img_buffer)


func = mod.get_function('mean_filter')
func(gray_gpu, res_gpu, np.int32(10), np.int32(h), np.int32(w), block=(32,32,1), grid=(120, 68))

cuda.memcpy_dtoh(res_img_buffer, res_gpu)

print('Original')
cv2_imshow(dataset.img0)
print('')
cv2_imshow(gray_img)
cv2_imshow(res_img_buffer)


In [ ]:
rgb = dataset.img0.astype(np.float32)
h,w = rgb.shape[0:2]
gray = np.zeros([h,w]).astype(np.float32)
H = np.zeros([6,6]).astype(np.float32)

rgb_gpu = cuda.mem_alloc(rgb.nbytes)
gray_gpu = cuda.mem_alloc(gray.nbytes)
H_gpu = cuda.mem_alloc(H.nbytes)

cuda.memcpy_htod(rgb_gpu, rgb)
cuda.memcpy_htod(gray_gpu, gray)
cuda.memcpy_htod(H_gpu, H)

func = mod.get_function('rgb2gray_gpu')
func(rgb_gpu, gray_gpu, H_gpu, np.int32(h), np.int32(w), block=(32,32,1), grid=(120, 68))

cuda.memcpy_dtoh(gray, gray_gpu)

cv2_imshow(gray)

In [ ]:
pnts1, pnts2, matches, kp0, kp1 = feature_matching(dataset.img0, dataset.img1)
F, mask = estimate_F_RANSAC(pnts1, pnts2, 32, 3000, 0.001)

print('Number of Inliers of current Fundamental Matrix', mask.sum())
    
_, H0, H1 = cv2.stereoRectifyUncalibrated(pnts1[mask], pnts2[mask], F, imgSize=(w, h))
# rect_img0 = cv2.warpPerspective(dataset.img0, H1, (w,h))
# rect_img1 = cv2.warpPerspective(dataset.img1, H2, (w,h))

img0 = dataset.img0.astype(np.float32)
res_img0 = np.zeros_like(img0)
H0 = H0.astype(np.float32)
img1 = dataset.img1.astype(np.float32)
res_img1 = np.zeros_like(img1)
H1 = H1.astype(np.float32)

img0_gpu = cuda.mem_alloc(img0.nbytes)
res0_gpu = cuda.mem_alloc(res_img0.nbytes)
H0_gpu = cuda.mem_alloc(H0.nbytes)

img1_gpu = cuda.mem_alloc(img1.nbytes)
res1_gpu = cuda.mem_alloc(res_img1.nbytes)
H1_gpu = cuda.mem_alloc(H1.nbytes)

cuda.memcpy_htod(img0_gpu, img0)
cuda.memcpy_htod(res0_gpu, res_img0)
cuda.memcpy_htod(H0_gpu, H0)

cuda.memcpy_htod(img1_gpu, img1)
cuda.memcpy_htod(res1_gpu, res_img1)
cuda.memcpy_htod(H1_gpu, H1)

func = mod.get_function('warpPerspectiveGpu')
func(img0_gpu, res0_gpu, H0_gpu, np.int32(h), np.int32(w), np.int32(3), block=(32,32,1), grid=(120, 68))
func(img1_gpu, res1_gpu, H1_gpu, np.int32(h), np.int32(w), np.int32(3), block=(32,32,1), grid=(120, 68))

cuda.memcpy_dtoh(res_img0, res0_gpu)
cuda.memcpy_dtoh(res_img1, res1_gpu)

res_img = np.concatenate([res_img0, res_img1], 1)
cv2_imshow(res_img)


In [ ]:
max_disp = np.int32(dataset.calib.ndisp)

disparity = np.zeros([h,w]).astype(np.int32) 
disparity_gpu = cuda.mem_alloc(disparity.nbytes)
cuda.memcpy_htod(disparity_gpu, disparity)
func = mod.get_function('compute_disparity')
func(res0_gpu, res1_gpu, disparity_gpu, np.int32(h), np.int32(w), max_disp, np.int32(9), block=(32,32,1), grid=(120, 68))

cuda.memcpy_dtoh(disparity, disparity_gpu)

focal_length = dataset.calib.cam0[0,0]*0.5 + dataset.calib.cam1[1,1]*0.5
baseline = dataset.calib.baseline

# Mark the zero disparities to -1 to prevent dividing zeros
# when computing depth
m = disparity == 0
disparity[m] = -1
disparity = disparity.reshape(h, w)

# Compute depth image
Depth = baseline*0.001*focal_length/(disparity)

# Change Negative depths to 0
Depth = np.maximum(Depth, 0)

# Clip disparity values using the bounding values provided
Disparity = np.clip(disparity, dataset.calib.vmin, dataset.calib.vmax)

# Normalize between 0~1
Disparity = (Disparity - Disparity.min())/(Disparity.max()-Disparity.min())

# 0 ~ 255
gray_disparity = (Disparity*255).astype(np.uint8)
# Convert to color heap map
color_disparity = cv2.applyColorMap(gray_disparity, cv2.COLORMAP_JET)

cv2.imwrite(dataset.name+'_disparity.png', gray_disparity)
cv2.imwrite(dataset.name+'_clr_disparity.png', color_disparity)

# Normalize between 0~1
Depth = (Depth - Depth.min())/(Depth.max() - Depth.min())

gray_depth = (Depth*255).astype(np.uint8)
color_depth = cv2.applyColorMap(gray_depth, cv2.COLORMAP_JET)

cv2.imwrite(dataset.name+'_depth.png', gray_depth)
cv2.imwrite(dataset.name+'_clr_depth.png', color_depth)



NameError: ignored